In [65]:
import struct
import sys
import numpy as np

def parse(f):
    data = f.read()
    image_data = np.zeros((96*96,))
    for i in range(96*96):
        image_data[i] = struct.unpack_from("<H", data, i*2)[0]
    return np.reshape(image_data, (96, 96), 'C') # use [y][x]
    
def stat(image_data):
    # image stats

    img_min = np.min(image_data)
    img_max = np.max(image_data)

    print("min", img_min, "max", img_max)

    # stddev

    print("std", np.std(image_data))

    avg = np.zeros((96,)) 
    for i in range(96):
        avg[i] = np.std(image_data[i, :])

    print(avg.mean())

In [66]:
empty = ["img_e0", "img_e1", "img_e2"]
fpimg = ["img_fp0", "img_fp1", "img_fp2"]

empty_data = []
for i in empty:
    with open(i, 'rb') as f:
        empty_data.append(parse(f))

fpimg_data = []
for i in fpimg:
    with open(i, 'rb') as f:
        fpimg_data.append(parse(f))

In [67]:
from matplotlib import pyplot as plt 
%matplotlib widget

In [68]:
for i in range(3):
    plt.figure(i+1)
    y = plt.hist(empty_data[i])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [69]:
for i in range(3):
    plt.figure(i+4)
    plt.hist(fpimg_data[i])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [70]:
# try to do the average thing

def empty_check(img):
    middle = img.mean()
    
    low_half = img[img < middle]
    high_half = img[img > middle]
    
    #print(middle, middle-np.max(low_half), np.min(high_half)-middle)
    distavg = np.mean([middle-np.max(low_half), np.min(high_half)-middle])
    print(distavg)
    
    stddev = np.std(img)
    print(stddev)
    
    # generally, the standard deviation seems to be lower for real images
    is_real = 0
    is_empty = 0
    
    if stddev < 850:
        is_real += 1
    if stddev > 2200:
        is_empty += 1
        
    # generally, the distavg is very low for real images
    if distavg < 200:
        is_real += 1
    if distavg > 1100:
        is_empty += 1
    
    print("real", is_real, "empty", is_empty)
    
    if is_real > is_empty:
        print("real")
    elif is_empty > is_real:
        print("empty")
    else:
        print("undecided")
    
    

In [71]:
for i in range(3):
    print("e", i)
    empty_check(empty_data[i])

e 0
2295.0
2900.4447351402846
real 0 empty 2
empty
e 1
2258.0
2859.711744869388
real 0 empty 2
empty
e 2
2167.0
2891.5263365594587
real 0 empty 2
empty


In [72]:
for i in range(3):
    print("f", i)
    empty_check(fpimg_data[i])

f 0
0.5
616.2800133202728
real 2 empty 0
real
f 1
0.5
531.5317568895721
real 2 empty 0
real
f 2
0.5
592.845492229728
real 2 empty 0
real


In [73]:
with open("/home/matthew/Downloads/img", "rb") as f:
    fpimg4 = parse(f)

In [74]:
empty_check(fpimg4)

0.5
696.8461397578617
real 2 empty 0
real


In [100]:
plt.close(12)
g = plt.figure(12)
plt.imshow(fpimg_data[0])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [112]:
# check to see what my histogram crap does

def x(data):
    sorted_pixels = np.unique(data)
    count = len(sorted_pixels)
    lvl1 = sorted_pixels[int(count * 0.08)]
    lvl2 = sorted_pixels[int(count * 0.92)]
    
    fig, (ax1, ax2) = plt.subplots(1, 2)
    
    real_img = np.zeros_like(data)
    for y in range(96):
        for x in range(96):
            value = data[y, x]
            if value < lvl1:
                continue
            elif lvl2 < value:
                real_img[y, x] = 255
            else:
                real_img[y, x] = (256 / (lvl2 - lvl1)) * (value - lvl1)
    
    ax1.imshow(real_img, cmap=plt.cm.binary)
    ax2.hist(real_img)

In [116]:
x(empty_data[2])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …